In [148]:
import os
from enum import Enum
import backtrader as bt
import argparse
import pandas as pd
import numpy as np
import time
from TestStrategy import TestStrategy

DT_COUNT = 0  # jupyter运行次数统计
DT_RESULT = None  # 回测大脑返回

In [247]:

def parse_args(pargs=None):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description='Sample for Order Target')

    parser.add_argument('--data', required=False,
                        default=DT_FILE_PATH,
                        help='Specific data to be read in')
    parser.add_argument('--dtformat', required=False, default=DT_DTFORMAT,
                        help='Ending date in data datetime format')
    parser.add_argument('--fromdate', required=False, default=DT_START,
                        help='Starting date in `dtformat` format')
    parser.add_argument('--todate', required=False, default=DT_END,
                        help='Ending date in `dtformat` format')
    parser.add_argument('--timeframe', required=False, default=DT_TIMEFRAME,
                        choices=['minutes', 'daily', 'weekly', 'monthly'],
                        help='重新采样到的时间范围')
    parser.add_argument('--compression', required=False, type=int, default=DT_COMPRESSION,
                        help='将 n 条压缩为 1, 最小周期为原数据周期')
    parser.add_argument('--opts', required=False, type=bool, default=DT_OPTS,
                        help='策略优化')
    parser.add_argument('--quantstats', required=False, type=int, default=DT_QUANTSTATS,
                        help='是否使用 quantstats 分析测试结果')
    parser.add_argument('--maxcpus', '-m', type=int, required=False, default=15,
                        help=('Number of CPUs to use in the optimization'
                              '\n  - 0 (default): 使用所有可用的 CPU\n   - 1 -> n: 使用尽可能多的指定\n'))
    parser.add_argument('--no-optdatas', action='store_true', required=False, help='优化中不优化数据预加载')
    parser.add_argument('--no-optreturn', action='store_true', required=False,
                        help='不要优化返回值以节省时间,这避免了回传大量生成的数据，例如指标在回溯测试期间生成的值')

    # Plot options
    parser.add_argument('--plot', '-p', nargs='?', required=False,
                        metavar='kwargs', const=True, default=DT_PLOT,
                        help=('绘制应用传递的任何 kwargs 的读取数据\n'
                              '\n''例如:\n''\n''  --plot style="candle" (to plot candles)\n'))
    parser.add_argument("-f", "--file", default="file")  # 接收这个-f参数
    if pargs is not None:
        return parser.parse_args(pargs)

    return parser.parse_args()


def runstrat(args=None):
    args = parse_args(args)
    dkwargs = dict()

    file_path = dt_start = dt_end = dt_format = dt_dtformat = dt_tmformat = ""

    if args.dtformat is not None:
        dt_format = args.dtformat
        dt_dtformat = dt_format[:dt_format.find('%d') + len('%d')]
        dt_tmformat = dt_format[dt_format.find('%H'):]
    if args.fromdate is not None:
        dt_start = bt.datetime.datetime.strptime(args.fromdate, dt_dtformat)
    if args.todate is not None:
        dt_end = bt.datetime.datetime.strptime(args.todate, dt_dtformat)
    if args.data is not None:
        file_path = args.data

    myQuant_ROOT = os.getcwd()[:os.getcwd().find("bt_backtrader\\") + len("bt_backtrader\\")]  # 获取项目中相对根路径
    file_path_abs = os.path.join(myQuant_ROOT, file_path)  # 文件路径
    print(file_path_abs)
    print("dt_format:", dt_format, "dt_start:", dt_start, "dt_end:", dt_end)
    if not os.path.exists(file_path_abs):
        raise Exception("数据源文件未找到！" + file_path_abs)

    dkwargs['fromdate'] = dt_start
    dkwargs['todate'] = dt_end
    dkwargs['dtformat'] = dt_format
    # dkwargs['tmformat'] = dt_tmformat

    print(dkwargs)
    # 加载数据
    df = pd.read_csv(filepath_or_buffer=file_path_abs,
                     # parse_dates={'datetime': ['date', 'time']},  # 日期和时间分开的情况
                     parse_dates=['datetime'],
                     index_col='datetime',
                     infer_datetime_format=True,
                     )

    # df.sort_values(by=["datetime"], ascending=True, inplace=True)  # 按日期先后排序
    # df.sort_values(by=["date", "time"], ascending=True, inplace=True)  # 按日期时间列先后排序

    # df.index = pd.to_datetime(df.date + ' ' + df.time, format=dt_format)  # 方式1: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df.date.astype(str) + ' ' + df.time.astype(str), format=dt_format)  # 方式2: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df['date'] + ' ' + df['time'], format=dt_format)  # 方式3: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df['date'], format=dt_dtformat) + pd.to_timedelta(df['time'])  # 方式4: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df.pop('date')) + pd.to_timedelta(df.pop('time'))  # 方式5: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df['date'].str.cat(df['time'], sep=' '), format=dt_format)  # 方式6: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # 截取时间段内样本数据
    df = df[dt_start:dt_end]
    # 增加一列openinterest
    df['openinterest'] = 0.00
    # 取出特定的列
    df = df[['open', 'high', 'low', 'close', 'volume']]
    # 列名修改成指定的
    df.rename(columns={"volume": "vol"}, inplace=True)

    tframes = dict(
        minutes=bt.TimeFrame.Minutes,
        daily=bt.TimeFrame.Days,
        weekly=bt.TimeFrame.Weeks,
        monthly=bt.TimeFrame.Months)

    # 使用pandas数据源创建交易数据集
    # data = bt.feeds.GenericCSVData(dataname=file_path, **dkwargs)
    data = (bt.feeds.PandasData(dataname=df, fromdate=dt_start, todate=dt_end))
    # 重采样到更大时间框架
    if args.timeframe and args.compression:
        data.resample(timeframe=tframes[args.timeframe], compression=args.compression)

    cerebro = bt.Cerebro(stdstats=False)

    cerebro.adddata(data)
    # 设置投资金额100000.0
    cerebro.broker.setcash(100000.0)

    # <editor-fold desc="折叠代码:交易手续费设置">
    cerebro.broker.setcommission(
        # 交易手续费，根据margin取值情况区分是百分比手续费还是固定手续费
        # commission=0.0015,
        commission=2.4,
        # 期货保证金，决定着交易费用的类型,只有在stocklike=False时起作用
        margin=0,
        # 乘数，盈亏会按该乘数进行放大
        mult=10.0,
        # 交易费用计算方式，取值有：
        # 1.CommInfoBase.COMM_PERC 百分比费用
        # 2.CommInfoBase.COMM_FIXED 固定费用
        # 3.None 根据 margin 取值来确定类型
        commtype=bt.CommInfoBase.COMM_FIXED,
        # 当交易费用处于百分比模式下时，commission 是否为 % 形式
        # True，表示不以 % 为单位，0.XX 形式；False，表示以 % 为单位，XX% 形式
        percabs=True,
        # 是否为股票模式，该模式通常由margin和commtype参数决定
        # margin=None或COMM_PERC模式时，就会stocklike=True，对应股票手续费；
        # margin设置了取值或COMM_FIXED模式时,就会stocklike=False，对应期货手续费
        stocklike=False,
        # 计算持有的空头头寸的年化利息
        # days * price * abs(size) * (interest / 365)
        interest=0.0,
        # 计算持有的多头头寸的年化利息
        interest_long=False,
        # 杠杆比率，交易时按该杠杆调整所需现金
        leverage=1.0,
        # 自动计算保证金
        # 如果 False, 则通过margin参数确定保证金
        # 如果automargin<0, 通过mult*price确定保证金
        # 如果automargin>0, 如果automargin*price确定保证金
        automargin=10 * 0.10,
        # 交易费用设置作用的数据集(也就是作用的标的)
        # 如果取值为None，则默认作用于所有数据集(也就是作用于所有assets)
        name=None
    )
    # </editor-fold>

    strats = None
    global DT_RESULT  # 申明要使用全局变量
    # 参数调优
    if args.opts:
        result = None
        # 为Cerebro引擎添加策略, 优化策略
        strats = cerebro.optstrategy(
            TestStrategy,
            rpp=range(DT_RPP[1], DT_RPP[2]),
            spp=range(DT_SPP[1], DT_SPP[2]),
            printlog=False,
        )
        # 添加分析指标
        cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timeReturn', timeframe=bt.TimeFrame.Years)  # 此分析器通过查看时间范围的开始和结束来计算回报
        cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")  # 使用对数方法计算的总回报、平均回报、复合回报和年化回报
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")  # 回撤
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")  # 夏普率
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="tradeAnalyzer")  # 提供有关平仓交易的统计信息（也保留未平仓交易的数量）
        # clock the start of the process
        tstart = time.perf_counter()
        # Run over everything
        # result = cerebro.run()
        DT_RESULT = result = cerebro.run(
            maxcpus=args.maxcpus,
            optdatas=not args.no_optdatas,  # optdatas（默认值：True)如果和优化（以及系统可以和使用），数据预加载将只在主进程中完成一次，以节省时间和资源。
            optreturn=not args.no_optreturn,  # optreturn（默认值：True)如果优化结果不是完整的对象（以及所有数据、指标、观察器等），而是具有以下属性的对象 在大多数情况下，只有分析器和哪些参数是评估策略性能所需的东西。如果需要对（例如）指标的生成值进行详细分析，请将其关闭
            # optreturn=False,
            # stdstats=False,
        )
        # clock the end of the process
        tend = time.perf_counter()
        # print out the result
        print('Time used:', str(tend - tstart))

        print("\n--------------- 分析结果 -----------------")

        # 每个策略实例的结果以列表的形式保存在列表中。
        # 优化运行模式下，返回值是列表的列表,内列表只含一个元素，即策略实例
        res_list = [[]]
        res_timereturn_title = []  # 列标题

        timeReturn = result[0][0].analyzers.timeReturn.get_analysis()  # timeReturn 分析引用
        for k, v in timeReturn.items():
            res_timereturn_title.append('{:%Y-%m}'.format(k))

        for x in result:
            trade = x[0].analyzers.tradeAnalyzer.get_analysis()  # 交易分析引用
            returns = x[0].analyzers.returns.get_analysis()  # 回报分析引用
            drawdown = x[0].analyzers.drawdown.get_analysis()  # 回撤分析引用
            sharpe = x[0].analyzers.sharpe.get_analysis()  # sharpe分析引用
            timeReturn = x[0].analyzers.timeReturn.get_analysis()  # timeReturn 分析引用

            row = [
                '{:2d}'.format(x[0].p.rpp),  # 参数
                '{:2d}'.format(x[0].p.spp),  # 参数
                '{:5.2f}'.format(returns['rtot'] * 100),  # 总复合回报
                '{:5.2f}'.format((trade['won']['total']) / (trade['lost']['total'] + trade['won']['total']) * 100),  # 胜率
                '{:5.2f}'.format(returns['rnorm100']),  # 以 100% 表示的年化归一化回报
                '{:5.2f}'.format(drawdown['max']['drawdown']),  # 最大回撤
                '{:5.5f}'.format(sharpe['sharperatio']),  # 夏普率
                '{:5.2f}'.format((((trade['pnl']['gross']['total']) - (trade['pnl']['net']['total'])) / (trade['pnl']['gross']['total'])) * 100),  # 手续费占比净盈亏百分比
                '{:4d}'.format(trade['total']['total']),  # 交易次数
                '{:8.2f}'.format(trade['pnl']['net']['total']),  # 帐户余额含手续费
            ]
            res_timereturn_row = list(timeReturn.values())  # 月度或年度复合回报,由参数timeframe=bt.TimeFrame.Months控制
            row.extend(res_timereturn_row)  # 月度复合回报
            # 变量是浮点数时,且<10时,%比显示保留5位整数1位小数且*100,>10时保留8位整数2位小数,不是浮点数时保留5位整数
            # row = [(('{:5.1f}' if abs(i) < 10 else '{:8.2f}') if isinstance(i, float) else '{:5}').format(i * 100 if (abs(i) < 10 and isinstance(i, float)) else i) for i in row]
            res_list.append(row)  # 添加到返回列表

        # print('res_list', type(res_list))
        # res_list = list(filter(None, res_list))  # 过滤空值

        # 结果转成dataframe
        columns = ['rpp', 'spp', 'rtot%', 'won%', 'rnorm%', 'maxDD%', 'sharpe', 'comm%', 'total', 'net']
        columns_all = columns.copy()
        columns_all.extend(res_timereturn_title)  # 将列标题添加到columns列表里
        res_df = pd.DataFrame(res_list, dtype='float', columns=columns_all)
        res_df.index.name = 'id'  # 设置索引名称
        res_df = res_df.dropna(how='any', axis=0)  # 删除所有带NaN的行
        res_df.loc[:, ['rpp', 'spp']].astype(int)
        res_df[['rtot%']] = res_df[['rtot%']].astype('float')
        # res_df.loc[:, ['rpp', 'spp', 'rtot%']].apply(pd.to_numeric, errors='ignore', axis=1)
        # res_df.loc[:, ['rpp', 'spp']].apply(pd.to_numeric, downcast='signed', axis=1)

        pd.set_option('precision', 3)  # 显示小数点后的位数
        # pd.set_option('display.min_rows', 300)  # 确定显示的部分有多少行
        pd.set_option('display.max_rows', 300)  # 确定显示的部分有多少行
        pd.set_option('display.max_columns', 20)  # 确定显示的部分有多少行
        pd.set_option('display.float_format', '{:,.2f}'.format)  # 逗号格式化大值数字,设置数据精度
        pd.set_option('expand_frame_repr', False)  # True就是可以换行显示。设置成False的时候不允许换行
        pd.set_option('display.width', 180)  # 设置打印宽度(**重要**)

        res_df = res_df.sort_values(by='rtot%', ascending=False)  # 按总复合回报排序
        res_df[res_df.columns[:5]].info()  # 显示前几列的数据类型
        print(res_df.loc[:, columns])  # 显示指定列
        res_df.to_csv('result.csv', sep='\t', float_format='%.2f')  # 保存分析数据到文件
    # 回测分析
    else:
        # 添加观测器,绘制时显示
        cerebro.addobserver(bt.observers.Broker)
        cerebro.addobserver(bt.observers.Trades)
        cerebro.addobserver(bt.observers.BuySell)
        cerebro.addobserver(bt.observers.DrawDown)
        # cerebro.addobserver(bt.observers.TimeReturn)
        # 添加分析指标
        cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='annualReturn')  # 返回年初至年末的年度收益率
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawDown')  # 计算最大回撤相关指标
        cerebro.addanalyzer(bt.analyzers.Returns, _name='returns', tann=252)  # 计算年化收益：日度收益
        cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyFolio')  # 添加PyFolio
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpeRatio', timeframe=bt.TimeFrame.Days, annualize=True, riskfreerate=0)  # 计算年化夏普比率：日度收益
        cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='sharpeRatio_A')
        cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timeReturn', )  # 添加收益率时序

        # 添加策略和参数
        cerebro.addstrategy(TestStrategy, rpp=DT_RPP[0], spp=DT_SPP[0], printlog=DT_PRINTLOG)

        # 引擎运行前打印期出资金
        print('组合期初资金: %.2f' % cerebro.broker.getvalue())
        # 启动回测
        DT_RESULT = result = cerebro.run()
        # 引擎运行后打期末资金
        print('组合期末资金: %.2f' % cerebro.broker.getvalue())
        # 提取结果
        print("\n--------------- 累计收益率 -----------------")
        annualReturn = result[0].analyzers.annualReturn.get_analysis()
        print(" Cumulative Return: {:.2f}".format(sum(annualReturn.values())))
        print("\n--------------- 年度收益率 -----------------")
        # print(' 收益率k,v', get_analysis.items())
        for k, v in annualReturn.items():
            print((" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v), end='')
        print("\n--------------- 最大回撤 -----------------")
        drawDown = result[0].analyzers.drawDown.get_analysis()
        for k, v in drawDown.items():
            if not isinstance(v, dict):
                t = (" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v)
                print(t, end='')
            else:
                for kk, vv in v.items():
                    t = (" [{:},{:.2f}]" if isinstance(vv, float) else " [{:},{:}]").format(kk, vv)
                    print(t, end='')
        print("\n--------------- 年化收益：日度收益 -----------------")
        returns = result[0].analyzers.returns.get_analysis()
        for k, v in returns.items():
            print((" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v), end='')
        print("\n--------------- 年化夏普比率：日度收益 -----------------")
        sharpeRatio = (result[0].analyzers.sharpeRatio.get_analysis())
        for k, v in sharpeRatio.items():
            print((" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v), end='')
        print("\n--------------- SharpeRatio_A -----------------")
        sharpeRatio_A = result[0].analyzers.sharpeRatio_A.get_analysis()
        for k, v in sharpeRatio_A.items():
            print((" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v), end='')
        print("\n--------------- test end -----------------")
    # 绘图
    if args.plot and not args.opts:
        pkwargs = dict(style='bar')
        if args.plot is not True:  # evals to True but is not True
            npkwargs = eval('dict(' + args.plot + ')')  # args were passed
            pkwargs.update(npkwargs)

        cerebro.plot(volume=False, **pkwargs)  # 绘图BT观察器结果

        # 结合pyfolio工具 计算并绘制收益评价指标
        import pyfolio as pf
        # 绘制图形
        import matplotlib.pyplot as plt
        plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
        import matplotlib.ticker as ticker  # 导入设置坐标轴的模块
        # plt.style.use('seaborn')
        plt.style.use('dark_background')

        # 提取收益序列
        pnl = pd.Series(result[0].analyzers.timeReturn.get_analysis())
        # 计算累计收益
        cumulative = (pnl + 1).cumprod()
        # 计算回撤序列
        max_return = cumulative.cummax()
        drawdown = (cumulative - max_return) / max_return
        # 按年统计收益指标
        perf_stats_year = pnl.groupby(pnl.index.to_period('y')).apply(lambda data: pf.timeseries.perf_stats(data)).unstack()
        # 统计所有时间段的收益指标
        perf_stats_all = pf.timeseries.perf_stats(pnl).to_frame(name='all')
        perf_stats = pd.concat([perf_stats_year, perf_stats_all.T], axis=0)
        perf_stats_ = round(perf_stats, 4).reset_index()

        fig, (ax0, ax1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 4]}, figsize=(24, 16))
        cols_names = ['date', 'Annual\nreturn', 'Cumulative\nreturns', 'Annual\n volatility',
                      'Sharpe\nratio', 'Calmar\nratio', 'Stability', 'Max\ndrawdown',
                      'Omega\nratio', 'Sortino\nratio', 'Skew', 'Kurtosis', 'Tail\nratio',
                      'Daily value\nat risk']
        # 绘制表格
        ax0.set_axis_off()  # 除去坐标轴
        table = ax0.table(cellText=perf_stats_.values,
                          bbox=(0, 0, 1, 1),  # 设置表格位置， (x0, y0, width, height)
                          rowLoc='left',  # 行标题居中
                          cellLoc='left',
                          colLabels=cols_names,  # 设置列标题
                          colLoc='left',  # 列标题居中
                          edges='open'  # 不显示表格边框
                          )
        table.set_fontsize(13)

        # 绘制累计收益曲线
        ax2 = ax1.twinx()
        ax1.yaxis.set_ticks_position('right')  # 将回撤曲线的 y 轴移至右侧
        ax2.yaxis.set_ticks_position('left')  # 将累计收益曲线的 y 轴移至左侧
        # 绘制回撤曲线
        drawdown.plot.area(ax=ax1, label='drawdown (right)', rot=0, alpha=0.3, fontsize=13, grid=False)
        # 绘制累计收益曲线
        cumulative.plot(ax=ax2, color='#F1C40F', lw=2.0, label='cumret (left)', rot=0, fontsize=13, grid=False)
        # 不然 x 轴留有空白
        ax2.set_xbound(lower=cumulative.index.min(), upper=cumulative.index.max())
        # 主轴定位器：每 5 个月显示一个日期：根据具体天数来做排版
        ax2.xaxis.set_major_locator(ticker.MultipleLocator(120))
        # 同时绘制双轴的图例
        h1, l1 = ax1.get_legend_handles_labels()
        h2, l2 = ax2.get_legend_handles_labels()

        plt.legend(h1 + h2, l1 + l2, fontsize=12, loc='upper left', ncol=1)
        fig.tight_layout()  # 规整排版
        plt.show()
        # 结合pyfolio工具 计算并绘制收益评价指标
    # 回测分析保存到文件
    if args.quantstats and not args.opts:
        # 使用quantstats 分析工具并保存到HTML文件
        portfolio_stats = result[0].analyzers.getbyname('pyFolio')
        returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
        returns.index = returns.index.tz_convert(None)
        import quantstats
        # quantstats.reports.html(returns, output='stats.html', title=DT_FILE_PATH)  # 将分析指标保存到HTML文件
        # 将分析指标保存到HTML文件
        quantstats.reports.html(returns, output='stats.html',
                                title=DT_FILE_PATH + 'rpp:{:} spp:{:} count:{:}'.format(DT_RPP[0], DT_SPP[0], DT_COUNT))
        print("quantstats 测试分析结果已保存至目录所在文件 quantstats-tearsheet.html")
        # 使用quantstats 分析工具并保存到HTML文件

In [250]:
DT_FILE_PATH = "datas\\SQRB13-5m-20121224-20220330.csv"
DT_DTFORMAT = '%Y-%m-%d %H:%M:%S'
DT_START, DT_END = '2012-01-01', '2016-02-01'
DT_TIMEFRAME = 'minutes'  # 重采样更大时间周期
DT_COMPRESSION = 15  # 合成周期的bar数
DT_PRINTLOG = False  # 是否打印日志
DT_PLOT = False  # 是否绘图,还可提供绘图参数:'style="candle"'
DT_QUANTSTATS = True  # 是否使用 quantstats 分析测试结果
DT_OPTS = True  # 是否参数调优
DT_RPP = [5, 1, 20]  # 参数
DT_SPP = [17, 10, 20]  # 参数

In [251]:
"""-------主函数---------"""
if __name__ == '__main__':
    runstrat()

D:\study\python-demo\myQuant\bt_backtrader\datas\SQRB13-5m-20121224-20220330.csv
dt_format: %Y-%m-%d %H:%M:%S dt_start: 2012-01-01 00:00:00 dt_end: 2016-02-01 00:00:00
{'fromdate': datetime.datetime(2012, 1, 1, 0, 0), 'todate': datetime.datetime(2016, 2, 1, 0, 0), 'dtformat': '%Y-%m-%d %H:%M:%S'}
Time used: 128.79378819999692

--------------- 分析结果 -----------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 155 to 47
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rpp     190 non-null    float64
 1   spp     190 non-null    float64
 2   rtot%   190 non-null    float64
 3   won%    190 non-null    float64
 4   rnorm%  190 non-null    float64
dtypes: float64(5)
memory usage: 8.9 KB
      rpp   spp   rtot%  won%  rnorm%  maxDD%  sharpe    comm%  total        net
id                                                                              
155 16.00 14.00  170.87 24.51    0.01   44.46    0.65     2.63 103

In [230]:
# 使用quantstats 分析工具并保存到HTML文件
global DT_RESULT, DT_COUNT  # 申明要使用全局变量
DT_COUNT += 1
portfolio_stats = DT_RESULT[0].analyzers.getbyname('pyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)
import quantstats

quantstats.reports.html(
    returns, output='stats.html',
    title=DT_FILE_PATH + 'rpp:{:} spp:{:} count:{:}'
        .format(DT_RPP[0], DT_SPP[0], DT_COUNT))  # 将分析指标保存到HTML文件
print("quantstats 测试分析结果已保存至目录所在文件 quantstats-tearsheet.html")
# 使用quantstats 分析工具并保存到HTML文件

AttributeError: 'list' object has no attribute 'analyzers'

In [41]:
import pandas as pd

df = pd.read_csv(filepath_or_buffer='result.csv',
                 sep='\t')
print(df.info)

<bound method DataFrame.info of     Unnamed: 0  rpp   spp  rtot%  won%  rnorm%  maxDD%  sharpe   comm%  total  \
0           74  8.0  13.0    0.8  11.2     0.0   55.85    14.0  109.20  171.0   
1           85  9.0  14.0    1.7  11.3     0.0   57.66     6.0   90.30  151.0   
2           90  9.0  19.0    6.2  13.1     0.0   70.53    16.3   60.20  108.0   
3           82  9.0  11.0    7.5  12.4     0.0   54.07    17.2   58.10  194.0   
4           13  2.0  12.0   -3.6  16.1    -0.0   67.97   -14.6  129.40  206.0   
5           54  6.0  13.0   -6.7  10.2    -0.0   71.24    22.3 -360.70  177.0   
6            4  1.0  13.0   14.8  19.3     0.1   83.04    23.9   48.10  193.0   
7           23  3.0  12.0   24.4  15.2     0.2   71.03    29.0   35.10  199.0   
8           14  2.0  13.0   28.2  19.0     0.2   83.80    30.3   40.70  185.0   
9           42  5.0  11.0   42.7  13.3     0.3   59.13    34.4   25.30  211.0   
10          55  6.0  14.0   45.2  12.0     0.3   66.11    40.8   15.10  159.0

In [42]:
df = df.sort_values('rtot%')
pd.set_option('display.min_rows', 300)  # 确定显示的最小行有多少
print(df.loc[:, :'rtot%'].head(100))

    Unnamed: 0  rpp   spp  rtot%
73          47  5.0  16.0 -428.7
72          27  3.0  16.0 -420.7
71          10  1.0  19.0 -420.5
68          17  2.0  16.0 -419.4
67          20  2.0  19.0 -414.7
66           7  1.0  16.0 -414.5
65           9  1.0  18.0 -412.1
58          57  6.0  16.0 -344.5
57          37  4.0  16.0 -319.0
55          19  2.0  18.0 -295.6
53          67  7.0  16.0 -250.7
52          78  8.0  17.0 -249.7
50          48  5.0  17.0 -235.2
48          38  4.0  17.0 -220.3
45          36  4.0  15.0 -212.7
44          28  3.0  17.0 -211.4
43          18  2.0  17.0 -194.3
42           8  1.0  17.0 -186.8
40          26  3.0  15.0 -166.1
39          16  2.0  15.0 -165.3
38          59  6.0  18.0 -164.2
37          40  4.0  19.0 -163.5
36           6  1.0  15.0 -163.3
35          56  6.0  15.0 -159.3
34          46  5.0  15.0 -158.1
33          66  7.0  15.0 -150.7
32          89  9.0  18.0 -143.9
30          87  9.0  16.0 -138.0
29          88  9.0  17.0 -135.6
27        

In [246]:
print("\n--------------- 分析结果 -----------------")

# 每个策略实例的结果以列表的形式保存在列表中。
# 优化运行模式下，返回值是列表的列表,内列表只含一个元素，即策略实例
res_list = [[]]
res_timereturn_title = []  # 列标题

global DT_RESULT, DT_COUNT  # 申明要使用全局变量
result = DT_RESULT
timeReturn = result[0][0].analyzers.timeReturn.get_analysis()  # timeReturn 分析引用
for k, v in timeReturn.items():
    res_timereturn_title.append('{:%Y-%m}'.format(k))

for x in result:
    trade = x[0].analyzers.tradeAnalyzer.get_analysis()  # 交易分析引用
    returns = x[0].analyzers.returns.get_analysis()  # 回报分析引用
    drawdown = x[0].analyzers.drawdown.get_analysis()  # 回撤分析引用
    sharpe = x[0].analyzers.sharpe.get_analysis()  # sharpe分析引用
    timeReturn = x[0].analyzers.timeReturn.get_analysis()  # timeReturn 分析引用

    row = [
        '{:2d}'.format(x[0].p.rpp),  # 参数
        '{:2d}'.format(x[0].p.spp),  # 参数
        '{:5.2f}'.format(returns['rtot'] * 100),  # 总复合回报
        '{:5.2f}'.format((trade['won']['total']) / (trade['lost']['total'] + trade['won']['total']) * 100),  # 胜率
        '{:5.2f}'.format(returns['rnorm100']),  # 以 100% 表示的年化归一化回报
        '{:5.2f}'.format(drawdown['max']['drawdown']),  # 最大回撤
        '{:5.5f}'.format(sharpe['sharperatio']),  # 夏普率
        '{:5.2f}'.format((((trade['pnl']['gross']['total']) - (trade['pnl']['net']['total'])) / (trade['pnl']['gross']['total'])) * 100),  # 手续费占比净盈亏百分比
        '{:4d}'.format(trade['total']['total']),  # 交易次数
        '{:8.2f}'.format(trade['pnl']['net']['total']),  # 帐户余额含手续费
    ]
    res_timereturn_row = list(timeReturn.values())  # 月度或年度复合回报,由参数timeframe=bt.TimeFrame.Months控制
    row.extend(res_timereturn_row)  # 月度复合回报
    # 变量是浮点数时,且<10时,%比显示保留5位整数1位小数且*100,>10时保留8位整数2位小数,不是浮点数时保留5位整数
    # row = [(('{:5.1f}' if abs(i) < 10 else '{:8.2f}') if isinstance(i, float) else '{:5}').format(i * 100 if (abs(i) < 10 and isinstance(i, float)) else i) for i in row]
    res_list.append(row)  # 添加到返回列表

# print('res_list', type(res_list))
res_list = list(filter(None, res_list))  # 过滤空值

# 结果转成dataframe
columns = ['rpp', 'spp', 'rtot%', 'won%', 'rnorm%', 'maxDD%', 'sharpe', 'comm%', 'total', 'net']
columns_all = columns.copy()
columns_all.extend(res_timereturn_title)  # 将月标题添加到columns列表里
res_df = pd.DataFrame(res_list, dtype='float', columns=columns_all)
res_df.index.name = 'id'  # 设置索引名称
# res_df[['rpp', 'spp']] = res_df[['rpp', 'spp']].astype(int)   # 转换指定类数据类型
res_df.loc[:, ['rpp', 'spp']].astype(int)

pd.set_option('precision', 3)  # 显示小数点后的位数
# pd.set_option('display.min_rows', 300)  # 确定显示的部分有多少行
pd.set_option('display.max_rows', 300)  # 确定显示的部分有多少行
pd.set_option('display.max_columns', 20)  # 确定显示的部分有多少行
pd.set_option('display.float_format', '{:,.2f}'.format)  # 逗号格式化大值数字,设置数据精度
pd.set_option('expand_frame_repr', False)  # True就是可以换行显示。设置成False的时候不允许换行
pd.set_option('display.width', 180)  # 设置打印宽度(**重要**)
res_df = res_df.sort_values(by='rtot%', ascending=False)  # 按总复合回报排序
# res_df[res_df.columns[:5]].info()  # 显示前几列的数据类型
print(res_df.loc[:, columns])  # 显示指定列
res_df.to_csv('result.csv', sep='\t', float_format='%.2f')  # 保存分析数据到文件


--------------- 分析结果 -----------------
    rpp   spp  rtot%  won%  rnorm%  maxDD%  sharpe  comm%  total        net
id                                                                         
4  1.00 14.00  11.94 16.67    0.03   30.82    0.45  -5.21   7.00  -4,650.40
5  1.00 15.00  11.77 16.67    0.03   30.88    0.44  -4.99   7.00  -4,850.40
30 4.00 10.00   9.42 10.00    0.02   25.33    0.40 -38.96  11.00  -1,181.20
31 4.00 11.00   9.37 11.11    0.02   25.14    0.41 -33.55  10.00  -1,242.00
15 2.00 15.00   8.49 16.67    0.02   29.55    0.32  -4.32   7.00  -5,216.00
14 2.00 14.00   7.44 16.67    0.02   29.12    0.30  -3.44   7.00  -6,351.20
16 2.00 16.00   7.11 20.00    0.02   29.42    0.29  -4.02   6.00  -6,709.20
17 2.00 17.00   7.11 20.00    0.02   29.42    0.29  -4.02   6.00  -6,709.20
6  1.00 16.00   5.38 20.00    0.01   30.75    0.24  -2.89   6.00 -11,112.00
7  1.00 17.00   5.38 20.00    0.01   30.75    0.24  -2.89   6.00 -11,112.00
9  1.00 19.00   4.01 25.00    0.01   35.88    0.